In [ ]:
!nvidia-smi

Sun Jul  9 18:18:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import re

In [ ]:
  data = pd.read_csv('/content/drive/MyDrive/W3DEV/googleplaystore.csv')
  data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [ ]:
data = data.drop(['App', 'Last Updated', 'Current Ver', 'Android Ver'], axis=1)
data

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity
...,...,...,...,...,...,...,...,...,...
10836,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education
10837,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education
10838,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical
10839,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference


In [ ]:
data = data.drop(data.query('Rating>5').index)
data

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity
...,...,...,...,...,...,...,...,...,...
10836,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education
10837,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education
10838,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical
10839,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference


In [ ]:
data['Size'] = data['Size'].replace('Varies with device', np.nan)
data['Size'] = data['Size'].str.replace('M', '')
data['Size'] = data['Size'].str.replace('k', '').astype(float)
data['Size'] = data['Size'].fillna(data['Size'].mean())
data

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,ART_AND_DESIGN,4.1,159,19.00000,"10,000+",Free,0,Everyone,Art & Design
1,ART_AND_DESIGN,3.9,967,14.00000,"500,000+",Free,0,Everyone,Art & Design;Pretend Play
2,ART_AND_DESIGN,4.7,87510,8.70000,"5,000,000+",Free,0,Everyone,Art & Design
3,ART_AND_DESIGN,4.5,215644,25.00000,"50,000,000+",Free,0,Teen,Art & Design
4,ART_AND_DESIGN,4.3,967,2.80000,"100,000+",Free,0,Everyone,Art & Design;Creativity
...,...,...,...,...,...,...,...,...,...
10836,FAMILY,4.5,38,53.00000,"5,000+",Free,0,Everyone,Education
10837,FAMILY,5.0,4,3.60000,100+,Free,0,Everyone,Education
10838,MEDICAL,NaN,3,9.50000,"1,000+",Free,0,Everyone,Medical
10839,BOOKS_AND_REFERENCE,4.5,114,37.03544,"1,000+",Free,0,Mature 17+,Books & Reference


In [ ]:
data['Installs'] = data['Installs'].apply(lambda x: int(re.sub('[^\d.]', '', x.strip())) if isinstance(x, str) else np.nan)

In [ ]:
data['Price'] = data['Price'].apply(lambda x: float(re.sub('[^\d.]', '', x)) if isinstance(x, str) else np.nan)
data

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,ART_AND_DESIGN,4.1,159,19.00000,10000,Free,0.0,Everyone,Art & Design
1,ART_AND_DESIGN,3.9,967,14.00000,500000,Free,0.0,Everyone,Art & Design;Pretend Play
2,ART_AND_DESIGN,4.7,87510,8.70000,5000000,Free,0.0,Everyone,Art & Design
3,ART_AND_DESIGN,4.5,215644,25.00000,50000000,Free,0.0,Teen,Art & Design
4,ART_AND_DESIGN,4.3,967,2.80000,100000,Free,0.0,Everyone,Art & Design;Creativity
...,...,...,...,...,...,...,...,...,...
10836,FAMILY,4.5,38,53.00000,5000,Free,0.0,Everyone,Education
10837,FAMILY,5.0,4,3.60000,100,Free,0.0,Everyone,Education
10838,MEDICAL,NaN,3,9.50000,1000,Free,0.0,Everyone,Medical
10839,BOOKS_AND_REFERENCE,4.5,114,37.03544,1000,Free,0.0,Mature 17+,Books & Reference


In [ ]:
data = pd.get_dummies(data, columns=['Category', 'Type', 'Content Rating', 'Genres'])
data

,Rating,Reviews,Size,Installs,Price,Category_ART_AND_DESIGN,Category_AUTO_AND_VEHICLES,Category_BEAUTY,Category_BOOKS_AND_REFERENCE,Category_BUSINESS,...,Genres_Tools;Education,Genres_Travel & Local,Genres_Travel & Local;Action & Adventure,Genres_Trivia,Genres_Trivia;Education,Genres_Video Players & Editors,Genres_Video Players & Editors;Creativity,Genres_Video Players & Editors;Music & Video,Genres_Weather,Genres_Word
0,4.1,159,19.00000,10000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.9,967,14.00000,500000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4.7,87510,8.70000,5000000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.5,215644,25.00000,50000000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.3,967,2.80000,100000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,4.5,38,53.00000,5000,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10837,5.0,4,3.60000,100,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10838,NaN,3,9.50000,1000,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10839,4.5,114,37.03544,1000,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data = data.dropna()
data

,Rating,Reviews,Size,Installs,Price,Category_ART_AND_DESIGN,Category_AUTO_AND_VEHICLES,Category_BEAUTY,Category_BOOKS_AND_REFERENCE,Category_BUSINESS,...,Genres_Tools;Education,Genres_Travel & Local,Genres_Travel & Local;Action & Adventure,Genres_Trivia,Genres_Trivia;Education,Genres_Video Players & Editors,Genres_Video Players & Editors;Creativity,Genres_Video Players & Editors;Music & Video,Genres_Weather,Genres_Word
0,4.1,159,19.00000,10000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.9,967,14.00000,500000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4.7,87510,8.70000,5000000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.5,215644,25.00000,50000000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.3,967,2.80000,100000,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,4.0,7,2.60000,500,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10836,4.5,38,53.00000,5000,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10837,5.0,4,3.60000,100,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10839,4.5,114,37.03544,1000,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = data.drop('Rating', axis=1)
y = data['Rating']
X

,Reviews,Size,Installs,Price,Category_ART_AND_DESIGN,Category_AUTO_AND_VEHICLES,Category_BEAUTY,Category_BOOKS_AND_REFERENCE,Category_BUSINESS,Category_COMICS,...,Genres_Tools;Education,Genres_Travel & Local,Genres_Travel & Local;Action & Adventure,Genres_Trivia,Genres_Trivia;Education,Genres_Video Players & Editors,Genres_Video Players & Editors;Creativity,Genres_Video Players & Editors;Music & Video,Genres_Weather,Genres_Word
0,159,19.00000,10000,0.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,967,14.00000,500000,0.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,87510,8.70000,5000000,0.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,215644,25.00000,50000000,0.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,967,2.80000,100000,0.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,7,2.60000,500,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10836,38,53.00000,5000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10837,4,3.60000,100,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10839,114,37.03544,1000,0.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y

0        4.1
1        3.9
2        4.7
3        4.5
4        4.3
        ... 
10834    4.0
10836    4.5
10837    5.0
10839    4.5
10840    4.5
Name: Rating, Length: 9366, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,Reviews,Size,Installs,Price,Category_ART_AND_DESIGN,Category_AUTO_AND_VEHICLES,Category_BEAUTY,Category_BOOKS_AND_REFERENCE,Category_BUSINESS,Category_COMICS,...,Genres_Tools;Education,Genres_Travel & Local,Genres_Travel & Local;Action & Adventure,Genres_Trivia,Genres_Trivia;Education,Genres_Video Players & Editors,Genres_Video Players & Editors;Creativity,Genres_Video Players & Editors;Music & Video,Genres_Weather,Genres_Word
3990,349,11.00000,50000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1584,82145,30.00000,50000000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8292,74497,5.30000,5000000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8836,22382,6.60000,1000000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8718,42,3.90000,100,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6137,218,4.10000,10000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5509,1,49.00000,10,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5724,27,3.40000,5000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
912,388089,37.03544,10000000,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train

3990    4.5
1584    4.5
8292    4.0
8836    3.9
8718    4.6
       ... 
6137    3.3
5509    5.0
5724    4.2
912     3.7
8183    3.9
Name: Rating, Length: 7492, dtype: float64

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_train

3990    4.5
1584    4.5
8292    4.0
8836    3.9
8718    4.6
       ... 
6137    3.3
5509    5.0
5724    4.2
912     3.7
8183    3.9
Name: Rating, Length: 7492, dtype: float64

In [ ]:
y_pred = model.predict(X_test)
y_pred


array([4.24769424, 4.15157917, 4.20389491, ..., 4.21224407, 4.24839282,
       4.08477742])

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.24226495591694341
